# Discussion 13: Heavyball, Steepest Descent w/ Momentum

In this discussion, we will explore
* Heavyball/momentum method
* Nesterov accelerated gradient (NAG)

---

We have prescribed many algorithms for optimizing a function so far in this course, among them elaborate second order methods like Newton's method (or its usually more palatable quasi-Newton approximants), as well as various sophisticated techniques of selecting step sizes adaptively and optimally in certain situations. However, as we've said many times before, many of these methods require too much computation per iteration to be useful for very large functions, e.g. in machine learning applications, so often the simplest (yet, as we've shown, among the least efficient) method of **fixed step steepest descent** is the go-to in these cases. Today we discuss another method that has gained popularity in recent years due to the fact that its simplicity is comparable to steepest descent yet its performance is typically much better.

## The "Heavyball" Method

We have shown many examples of how steepest descent may take many thousands of iterations to converge even for relatively simple functions. One specific example of where steepest descent struggles is for quadratic functions where one variable is "scaled" much more than the other. Take the function

$$ f(x,y) = x^2 + 10y^2 $$

with $\mathbf{x}_0=(10,1)$ and $\alpha_k$ chosen by exact line search. Recall that for quadratic functions, the optimal value of $\alpha_k=\frac{\|\nabla f_k\|^2}{\|\nabla f_k\|_Q^2}$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
f = lambda x,y: x**2 + 10*y**2
df = lambda x,y: np.array([2*x, 20*y])
Q = np.array([[2,0],[0,20]])

plt.figure(figsize=(10, 5), dpi=150)
X = np.linspace(-10,10,300)
Y = np.linspace(-5,5,300)
Xmesh, Ymesh = np.meshgrid(X,Y)
Z = f(Xmesh,Ymesh)
CS = plt.contour(Xmesh, Ymesh, Z, np.arange(1,20)**2, cmap='jet')
plt.clabel(CS,inline_spacing=0,fmt='%d')
plt.axis([-10,10,-5,5])
plt.xlabel('x')
plt.ylabel('y')

x = np.array([10,1])
tol = 1e-3
max_steps = 1000
dx = df(x[0],x[1])
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    alpha = (dx@dx)/(dx@(Q@dx))   # optimal step size
    xnew = x - alpha*dx
    plt.arrow(x[0],x[1],-alpha*dx[0],-alpha*dx[1],color='k', head_width=.1,length_includes_head=True)
    x = xnew
    i += 1
    dx = df(x[0],x[1])

print(f'After {i} iterations, approximate minimum is {f(x[0],x[1])} at {x}')
plt.title('Steepest descent with optimal step size')
plt.show()

We see above that the steps of steepest descent appear perpendicular to each other, and indeed this can be proven since if $\phi'(\alpha)=0$, we have

$$ 0 = \phi'(\alpha) = \frac{d}{d\alpha}\big(f(\mathbf{x}_k - \alpha\nabla f_k)\big) = \nabla f_k^T\nabla f(\mathbf{x}_k - \alpha\nabla f_k) = \nabla f_k^T\nabla f_{k+1} = \mathbf{p}_k^T\mathbf{p}_{k+1}$$

and thus $\mathbf{p}_k$ and $\mathbf{p}_{k+1}$ are orthogonal. Since successive updates are orthogonal, every other update is essentially moving in the same direction, even though it is proven not to result in much decrease. In fact, the situation is worse if we fall back on choosing a fixed value of $\alpha$, in which case we see *extreme* oscillation:

In [ ]:
plt.figure(figsize=(10, 5), dpi=150)
X = np.linspace(-10,10,300)
Y = np.linspace(-5,5,300)
Xmesh, Ymesh = np.meshgrid(X,Y)
Z = f(Xmesh,Ymesh)
CS = plt.contour(Xmesh, Ymesh, Z, np.arange(1,20)**2, cmap='jet')
plt.clabel(CS,inline_spacing=0,fmt='%d')
plt.axis([-10,10,-5,5])
plt.xlabel('x')
plt.ylabel('y')

x = np.array([10,1])
tol = 1e-3
max_steps = 1000
dx = df(x[0],x[1])
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    alpha = .099   # fixed step size just under 0.1
    xnew = x - alpha*dx
    plt.arrow(x[0],x[1],-alpha*dx[0],-alpha*dx[1],color='k', head_width=.1,length_includes_head=True)
    x = xnew
    i += 1
    dx = df(x[0],x[1])

print(f'After {i} iterations, approximate minimum is {f(x[0],x[1])} at {x}')
plt.title('Steepest descent with fixed step size')
plt.show()

These steps are not orthogonal to each other, but they are exhibiting the same oscillating behavior. It would be nice if the iteration had some kind of "memory" of where it has been to avoid making the same mistakes multiple times. This is one big motivation behind the **heavyball method**, also referred to as **steepest descent with momentum**. The heavyball iteration is defined by the following:

$$ \begin{align*}
    \mathbf{p}_k &= -\alpha_k\nabla f_k \color{red}{+ \beta \mathbf{p}_{k-1}} \\
    \mathbf{x}_{k+1} &= \mathbf{x}_k + \mathbf{p}_k
\end{align*} $$

where $\beta\in[0,1]$ is called the *momentum parameter*. This update should look very familiar, as it is just the conjugate gradient update with a fixed $\beta$. In some sense, we can think of heavyball as somewhere in between steepest descent and conjugate gradient: heavyball attempts to force the new directions to be different than previous ones but doesn't go so far as requiring any kind of conjugacy. We should thus probably expect results somewhere in between steepest descent and conjugate gradient as well. Indeed we investigate below the same problem from above using the parameter $\beta=0.2$, first choosing the optimal step size, then choosing the same fixed step size as before:

In [ ]:
plt.figure(figsize=(10, 5), dpi=150)
X = np.linspace(-10,10,300)
Y = np.linspace(-5,5,300)
Xmesh, Ymesh = np.meshgrid(X,Y)
Z = f(Xmesh,Ymesh)
CS = plt.contour(Xmesh, Ymesh, Z, np.arange(1,20)**2, cmap='jet')
plt.clabel(CS,inline_spacing=0,fmt='%d')
plt.axis([-10,10,-5,5])
plt.xlabel('x')
plt.ylabel('y')

x = np.array([10,1])
tol = 1e-3
beta = 0.2    # momentum parameter
max_steps = 1000
dx = df(x[0],x[1])
pk = dx*0      # first update is just gradient descent
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    alpha = (dx@dx)/(dx@(Q@dx))   # optimal step size
    pk = -alpha*dx + beta*pk
    xnew = x + pk
    plt.arrow(x[0],x[1],xnew[0]-x[0],xnew[1]-x[1],color='k', head_width=.1,length_includes_head=True)
    x = xnew
    i += 1
    dx = df(x[0],x[1])

print(f'After {i} iterations, approximate minimum is {f(x[0],x[1])} at {x}')
plt.title('Heavyball with optimal step size')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5), dpi=150)
X = np.linspace(-10,10,300)
Y = np.linspace(-5,5,300)
Xmesh, Ymesh = np.meshgrid(X,Y)
Z = f(Xmesh,Ymesh)
CS = plt.contour(Xmesh, Ymesh, Z, np.arange(1,20)**2, cmap='jet')
plt.clabel(CS,inline_spacing=0,fmt='%d')
plt.axis([-10,10,-5,5])
plt.xlabel('x')
plt.ylabel('y')

x = np.array([10,1])
tol = 1e-3
beta = 0.2    # momentum parameter
max_steps = 1000
dx = df(x[0],x[1])
pk = dx*0      # first update is just gradient descent
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    alpha = .099   # fixed step size
    pk = -alpha*dx + beta*pk
    xnew = x + pk
    plt.arrow(x[0],x[1],xnew[0]-x[0],xnew[1]-x[1],color='k', head_width=.1,length_includes_head=True)
    x = xnew
    i += 1
    dx = df(x[0],x[1])

print(f'After {i} iterations, approximate minimum is {f(x[0],x[1])} at {x}')
plt.title('Heavyball with fixed step size')
plt.show()

The reason this is often called steepest descent *with momentum* is if two consecutive updates are in basically the same direction, this algorithm will move more quickly in that direction to try to reach the minimum faster. If the previous update was in a direction opposite to the current one, it has the effect of backing off and preventing overshooting or oscillation. The following image demonstrates the idea pictorially:

<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2017/03/07130330/momentum.png" width=30% />

For a great interactive animation of momentum (as well as a great explainer), see [here](https://distill.pub/2017/momentum/).

---

## Nesterov Accelerated Gradient (NAG)

The professor's notes do a great job of going into the detail of optimal values of $\beta$ (and $\alpha$) to choose for quadratic objective functions, and you will explore this idea more in a future homework, so I want to take this time to introduce a modification to heavyball which is also quite popular in machine learning applications, the **Nesterov accelerated gradient (NAG)** method.

Heavyball can often lead to better results than steepest descent, especially for functions with many suboptimal local minima or flat plateaus where the gradient almost vanishes. However, one of the drawbacks is that it can become *too* confident it's going in the correct direction, particularly if $\beta\sim1$, and overshoot the minimum like a ball rolling down a hill and only stopping when starting back uphill. This means the algorithm can introduce its own oscillations and prolong convergence. NAG is a very clever tweak of heavyball which still uses the idea of momentum, but looks *ahead* to see what the gradient *will be* at the *upcoming* point. That is, it first checks where the current momentum will take it, evaluates the gradient there, and updates the weights based on that prediction. This is akin to having a ball which knows where it's going, and thus it starts to put on the brakes *before* the upcoming hill. On the other hand, if the ball knows it will continue rolling down a hill, it hits the gas *even harder*.


Mathematically, the NAG update is defined by

$$ \begin{align*}
    \mathbf{p}_k &= -\nabla f(\color{red}{\mathbf{x}_k+\beta\mathbf{p}_{k-1}})+ \beta \mathbf{p}_{k-1} \\
    \mathbf{x}_{k+1} &= \mathbf{x}_k +\alpha_k \mathbf{p}_k
\end{align*} $$

a pictorial representation of which is compared to "regular" momentum below:

<img src="https://miro.medium.com/max/1600/1*hJSLxZMjYVzgF5A_MoqeVQ.jpeg" width=75% />

We implement NAG below on the same function as above, again with the optimal and fixed step sizes, and compare the results:

In [ ]:
plt.figure(figsize=(10, 5), dpi=150)
X = np.linspace(-10,10,300)
Y = np.linspace(-5,5,300)
Xmesh, Ymesh = np.meshgrid(X,Y)
Z = f(Xmesh,Ymesh)
CS = plt.contour(Xmesh, Ymesh, Z, np.arange(1,20)**2, cmap='jet')
plt.clabel(CS,inline_spacing=0,fmt='%d')
plt.axis([-10,10,-5,5])
plt.xlabel('x')
plt.ylabel('y')

x = np.array([10,1])
tol = 1e-3
beta = 0.2    # momentum parameter
max_steps = 1000
dx = df(x[0],x[1])
pk = dx*0      # first update is just gradient descent
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    alpha = (dx@dx)/(dx@(Q@dx))   # optimal step size
    pk = -alpha*df(x[0]+beta*pk[0],x[1]+beta*pk[1]) + beta*pk  # gradient "looks ahead"
    xnew = x + pk
    plt.arrow(x[0],x[1],xnew[0]-x[0],xnew[1]-x[1],color='k', head_width=.1,length_includes_head=True)
    x = xnew
    i += 1
    dx = df(x[0],x[1])

print(f'After {i} iterations, approximate minimum is {f(x[0],x[1])} at {x}')
plt.title('NAG with optimal step size')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5), dpi=150)
X = np.linspace(-10,10,300)
Y = np.linspace(-5,5,300)
Xmesh, Ymesh = np.meshgrid(X,Y)
Z = f(Xmesh,Ymesh)
CS = plt.contour(Xmesh, Ymesh, Z, np.arange(1,20)**2, cmap='jet')
plt.clabel(CS,inline_spacing=0,fmt='%d')
plt.axis([-10,10,-5,5])
plt.xlabel('x')
plt.ylabel('y')

x = np.array([10,1])
tol = 1e-3
beta = 0.2    # momentum parameter
max_steps = 1000
dx = df(x[0],x[1])
pk = dx*0      # first update is just gradient descent
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    alpha = .099   # fixed step size
    pk = -alpha*df(x[0]+beta*pk[0], x[1]+beta*pk[1]) + beta*pk  # gradient "looks ahead"
    xnew = x + pk
    plt.arrow(x[0],x[1],xnew[0]-x[0],xnew[1]-x[1],color='k', head_width=.1,length_includes_head=True)
    x = xnew
    i += 1
    dx = df(x[0],x[1])

print(f'After {i} iterations, approximate minimum is {f(x[0],x[1])} at {x}')
plt.title('NAG with fixed step size')
plt.show()

It turns out in this specific case that NAG performs worse than standard momentum, but that is only because of our choice of $\alpha$ and $\beta$ to be *just on the edge* of divergence. In typical applications where these parameters are not chosen so precariously, NAG typically has better results than standard (sometimes called classical) momentum.